In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, re
manager_id = "microsoft/Phi-3-mini-4k-instruct"
manager = AutoModelForCausalLM.from_pretrained( 
    manager_id,  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,  
)
manager_tok = AutoTokenizer.from_pretrained(manager_id) 

coder_id = 'Qwen/Qwen2.5-Coder-1.5B-Instruct'
coder = AutoModelForCausalLM.from_pretrained(coder_id, device_map='auto', torch_dtype="auto",trust_remote_code=True)
coder_tok = AutoTokenizer.from_pretrained(coder_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
history = []
def add_to_history(model, message):
    history.append(f"{model}:{message}\n")

def json_parser(gen_json):
    match = re.search(r'```(.*?)```', gen_json, re.DOTALL)
    if match:
        code_string = match.group(1).strip() 
        parsed_inst = code_string.replace('json', '')
        return parsed_inst

    else:
        print("No json found.") 
        print(gen_json)
        return None

def python_parser(gen_code):
    match = re.search(r'```(.*?)```', gen_code, re.DOTALL)
    if match:
        code_string = match.group(1).strip() 
        parsed_code = code_string.replace('python', '')
        return parsed_code 

    else:
        print("No code found.")
        return None

def manager_function(task):
    manager_prompt = "<|system|> Manager: Your job is to understand the task and just send the prompt over to the coder agent."
    manager_inputs = manager_prompt+f"\n<|user|>task prompt: {task}"+" Return the prompt in json format like {'Coder':'task prompt'}.<|end|>\n<|assistant|>"
    manager_inputs = manager_tok(manager_inputs, return_tensors='pt').to(manager.device)
    manager_outputs = manager.generate(**manager_inputs, max_new_tokens=100,)
    instruction = manager_tok.decode(manager_outputs[0], skip_special_tokens=True)
    add_to_history("Manager", task)
    return instruction

def coder_function(inst):
    parsed_inst = json_parser(inst)
    coder_prompt = "<|system|> Coder: Your job is to give the correct python code for the task prompt. The task prompt will be in json format."
    coder_inputs = coder_prompt+f"\n<|user|>{parsed_inst}\nAlso make sure that the code is executable in a ipykernel.<|end|>\n<|assistant|>"
    coder_inputs = coder_tok(coder_inputs, return_tensors='pt').to(coder.device)
    coder_outputs = coder.generate(**coder_inputs, max_new_tokens=500,)
    generated_code = coder_tok.decode(coder_outputs[0], skip_special_tokens=True)
    add_to_history("Coder", generated_code)
    return generated_code

def executor(gen_code):
    parsed_code = python_parser(gen_code)
    exec(parsed_code)

In [8]:
task = "Write a Python script using Selenium to google meet and schedule a meet for tomorrow 12pm"
inst = manager_function(task)
#print(inst)
code = coder_function(inst)
print(python_parser(code))
#executor(code)


import selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the WebDriver
driver = webdriver.Chrome()

# Navigate to Google Meet
driver.get("https://meet.google.com")

# Wait for the search bar to appear
search_bar = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "q"))
)

# Enter the search query
search_bar.send_keys("Meeting with John Doe")

# Submit the search
search_bar.submit()

# Wait for the search results to appear
results = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".GQo6Wd"))
)

# Select the first result (assuming it's the one you want)
first_result = results[0]
first_result.click()

# Wait for the calendar to open
calendar = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "jfk_calendar_0"))
)

# Click on the calendar button
calendar.cli

In [9]:
print('\n'.join(history))

Manager:Write a Python script using Selenium to scrape the homepage of Amazon.com.

Coder:<|system|> Coder: Your job is to give the correct python code for the task prompt. The task prompt will be in json format.
<|user|>

{

  "Coder": "Write a Python script using Selenium to scrape the homepage of Amazon.com."

}
Also make sure that the code is executable in a ipykernel.<|end|>
<|assistant|>```python
from selenium import webdriver
import time

# Initialize the web driver
driver = webdriver.Chrome()

# Open the Amazon home page
driver.get("https://www.amazon.com")

# Wait for a few seconds to see the page load
time.sleep(5)

# Print the title of the page
print(driver.title)

# Close the browser
driver.quit()
```

This script uses the Selenium library to open the Amazon home page, wait for it to load, and then print the title of the page. Finally, it closes the browser. The script is executable in an ipykernel because it imports the necessary libraries and functions from the Selenium m